##Introduction##

Infectious disease modelling and medical diagnostics, speech recognition and word tagging, document retrieval and categorization, and error-correcting codes are just some of the applications of probabilistic graphical models. Whether the discussion is centred on directed (Bayesian) or undirected (Markov Random Field) graphs, the problems of learning and inference have been tackled in various ways. In a graphical model framework, 'learning' refers to estimating parameters associated with a specific model (graphical) architecture, and sometimes to selecting the number of parameters to be used in modelling. By contrast, 'inference' is performed once the model has been built, and is used to make predictions about data and to quantify uncertainty on model parameters. While the focus of this work will be on approximate inference techniques, inference and learning procedures are intricately linked, and each should be kept in mind while the other is being performed.

##Inference techniques##

Inference on probabilistic graphical models can only be performed exactly when the graph structure is relatively limited. For example, the junction tree algorithm is an efficient algorithm for tree models (Bayesian networks that take the form of directed acyclic graphs). Another common form of exact inference on tree models is belief propagation (BP); however, if the graph is densely connected, BP can be computationally infeasible [1 - the book]. The generalization of BP to graphs that include cycles is called loopy belief propagation, which will be discussed later with an example. Both BP and loopy BP are variational methods.

On general graph structures, inference is computationally intractable due to the intractability of the normalization constant (also called the partition function, or the free energy). This constant often involves computation of a multidimensional integral, which cannot be evaluated analytically. Numerical methods are then required to arrive at a solution. 

These numerical methods can be stochastic or deterministic in origin. The most popular stochastic method is Markov chain Monte Carlo (MCMC), and its many variants. MCMC methods, though exact, can be slow over large graphical structures. As modelling and inference problems evolve to match the speed of data, approximate inference methods become more ubiquitous. Instead of sampling from the unknown, intractable posterior distribution (as in MCMC), variational methods construct a known distribution to approximate the posterior distribution. The parameters of the known distribution are then chosen to minimize the difference between the posterior and the variational distribution. In this way, we obtain a deterministic approximation to the posterior distribution, often in a faster way than with MCMC methods.

The most widely used class of variational inference algorithms are derived from the so-called 'mean-field approximation' from the statistical physics literature. MCMC also had its origins in statistical physics. 



In [8]:
#From http://www.bayespy.org/examples/gmm.html
import numpy as np

#First generate data in 4 clusters, with 100 individuals per cluster
#For example, y0 has mean [0,0] and identity covariance matrix
y0 = np.random.multivariate_normal([0, 0], [[1, 0], [0, 1]], size=100)
y1 = np.random.multivariate_normal([10, 10], [[1, 0], [0, 1]], size=100)
y2 = np.random.multivariate_normal([5, 5], [[1, 0], [0, 1]], size=100)
y3 = np.random.multivariate_normal([-2, 8], [[1, 0], [0, 1]], size=100)
y = np.vstack([y0, y1, y2, y3])
#y = y0

In [9]:
import bayespy.plot as bpplt
#, obs[:,1][50:99], 'bo',
#bpplt.pyplot.plot(y[:,0], y[:,1], 'rx')
bpplt.pyplot.plot(y[:,0][0:99], y[:,1][0:99], 'rx', y[:,0][100:199], y[:,1][100:199], 'kx', y[:,0][200:299], y[:,1][200:299], 'ko', y[:,0][300:399], y[:,1][300:399], 'ro')
bpplt.pyplot.ylabel('y')
bpplt.pyplot.xlabel('x')
bpplt.pyplot.title('Positions of individuals')


In [10]:
N = 400 #The number of data points
D = 2 #we're in 2 dimensions
K = 20 #guess there are a huge number of clusters, and let the algorithm pare it down.

#In a Gaussian mixture model the cluster assignment (label) follows a Categorical distribution
#and the prior for a cluster assignment follows a Dirichlet distribution (conjugate prior)

from bayespy.nodes import Dirichlet, Categorical
alpha = Dirichlet(1e-5*np.ones(K), name='alpha')
Z = Categorical(alpha, plates=(N,), name='z')

#The conjugate prior for a Gaussian (the mean vector) is Gaussian
#The conjugate prior for the precision matrix (inverse covariance matrix) is Wishart
from bayespy.nodes import Gaussian, Wishart

mu = Gaussian(np.zeros(D), 1e-5*np.identity(D), plates=(K,), name='mu')
Lambda = Wishart(D, 1e-5*np.identity(D), plates=(K,), name='Lambda')

print(Lambda)


Lambda ~ Wishart(n, A)
  n =
[ 2.]
  A =
[[[100000.      0.]
  [     0. 100000.]]]



In [11]:
#We know the data come from a Gaussian mixture model

from bayespy.nodes import Mixture

Y = Mixture(Z, Gaussian, mu, Lambda, name='Y') #This forms the mixture model
Z.initialize_from_random() #initialize the categorical random variable

#Form a VB object

from bayespy.inference import VB

Q = VB(Y, mu, Lambda, Z, alpha)

#Create observations of the data 

Y.observe(y)

# Run VB until convergence

Q.update(repeat=1000)





Iteration 1: loglike=-3.613117e+03 (0.014 seconds)
Iteration 2: loglike=-3.268603e+03 (0.021 seconds)
Iteration 3: loglike=-3.136195e+03 (0.014 seconds)
Iteration 4: loglike=-2.981444e+03 (0.018 seconds)
Iteration 5: loglike=-2.909490e+03 (0.019 seconds)
Iteration 6: loglike=-2.867081e+03 (0.018 seconds)
Iteration 7: loglike=-2.804979e+03 (0.015 seconds)
Iteration 8: loglike=-2.684564e+03 (0.021 seconds)
Iteration 9: loglike=-2.432585e+03 (0.016 seconds)
Iteration 10: loglike=-2.379510e+03 (0.013 seconds)
Iteration 11: loglike=-2.332555e+03 (0.020 seconds)
Iteration 12: loglike=-2.266174e+03 (0.015 seconds)
Iteration 13: loglike=-2.203237e+03 (0.011 seconds)
Iteration 14: loglike=-2.162500e+03 (0.018 seconds)
Iteration 15: loglike=-2.097966e+03 (0.013 seconds)
Iteration 16: loglike=-2.063617e+03 (0.012 seconds)
Iteration 17: loglike=-2.028999e+03 (0.011 seconds)
Iteration 18: loglike=-2.026357e+03 (0.015 seconds)
Iteration 19: loglike=-2.023448e+03 (0.013 seconds)
Iteration 20: loglike

In [17]:
#Visualize the clusters

bpplt.gaussian_mixture_2d(Y, alpha=alpha, scale=2)
bpplt.pyplot.show()

## The EM Algorithm ##

%http://www.sciencedirect.com/science/article/pii/S0167947311003963

"The expectation (E) step of the EM algorithm gives the posterior probabilities 
that the ith observation belongs to the kth component". 

"The M-step for Gaussian mixture models provides formulas for updating mixing proportions, mean vectors, and dispersion matrices"

"When clusters overlap substantially, it can be challenging for any method to provide a good solution" - I agree, but we won't run into that problem here.

A trivial example to start us off (two component Gaussian mixture model). Using the notation from Elements of Statistical Learning (2009):
I have data from 2 classes, and I want maximum likelihood estimates for $mu_1$, $mu_2$, $sigma^2_1$, $sigma^2_2$, and the mixing probabilities $\pi$

ImportError: No module named 'pypr'